In [ ]:
from munch import munchify
import sys
import os
import numpy as np

if os.path.basename(os.getcwd()) != "Splat-SLAM":
    os.chdir("..")

from thirdparty.glorie_slam import config
from thirdparty.gaussian_splatting.scene.gaussian_model import GaussianModel

%load_ext autoreload
%autoreload 2

In [3]:
config_path = "configs/LAC/data_collection_2.yaml"
cfg = config.load_config(config_path, "./configs/splat_slam.yaml")

In [4]:
model_params = munchify(cfg["mapping"]["model_params"])
use_spherical_harmonics = cfg["mapping"]["Training"]["spherical_harmonics"]
model_params.sh_degree = 3 if use_spherical_harmonics else 0
gaussians = GaussianModel(model_params.sh_degree, config=cfg)

In [5]:
ply_path = "/home/shared/outputs/guillemc/LAC/data_collection_2/point_cloud/100.ply"
gaussians.load_ply(ply_path)

In [ ]:
gaussians.get_xyz.shape

In [34]:
import viser
from viser import transforms as tf

from thirdparty.gaussian_splatting.utils.general_utils import reconstruct_sigma

In [ ]:
server = viser.ViserServer()
server.gui.configure_theme(dark_mode=True)
gui_reset_up = server.gui.add_button(
    "Reset up direction",
    hint="Set the camera control 'up' direction to the current camera's 'up'.",
)


@gui_reset_up.on_click
def _(event: viser.GuiEvent) -> None:
    client = event.client
    assert client is not None
    client.camera.up_direction = tf.SO3(client.camera.wxyz) @ np.array([0.0, -1.0, 0.0])

In [ ]:
gs_handle = server.scene.add_gaussian_splats(
    "splat map",
    centers=gaussians.get_xyz.cpu().detach().numpy(),
    rgbs=gaussians.get_features.cpu().detach().numpy().squeeze(),
    opacities=gaussians.get_opacity.cpu().detach().numpy(),
    covariances=reconstruct_sigma(gaussians.get_covariance()).cpu().detach().numpy(),
)